<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Merging-Data-from-Jackson-et-al-(2018)" data-toc-modified-id="Merging-Data-from-Jackson-et-al-(2018)-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Merging Data from Jackson et al (2018)</a></span><ul class="toc-item"><li><span><a href="#Parsing-Table-2" data-toc-modified-id="Parsing-Table-2-1.1"><span class="toc-item-num">1.1&nbsp;&nbsp;</span>Parsing Table 2</a></span></li><li><span><a href="#Parsing-Table-4" data-toc-modified-id="Parsing-Table-4-1.2"><span class="toc-item-num">1.2&nbsp;&nbsp;</span>Parsing Table 4</a></span></li><li><span><a href="#Merging-and-Unit-Correction" data-toc-modified-id="Merging-and-Unit-Correction-1.3"><span class="toc-item-num">1.3&nbsp;&nbsp;</span>Merging and Unit Correction</a></span></li></ul></li><li><span><a href="#Query-GAIA-DR2-and-2MASS" data-toc-modified-id="Query-GAIA-DR2-and-2MASS-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Query GAIA DR2 and 2MASS</a></span></li><li><span><a href="#Isoclassify-$R_*$-Calculations" data-toc-modified-id="Isoclassify-$R_*$-Calculations-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Isoclassify $R_*$ Calculations</a></span><ul class="toc-item"><li><span><a href="#Convert-Data-to-Isoclassify-readable-format" data-toc-modified-id="Convert-Data-to-Isoclassify-readable-format-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Convert Data to Isoclassify readable format</a></span></li><li><span><a href="#Executing-Isoclassify" data-toc-modified-id="Executing-Isoclassify-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Executing Isoclassify</a></span></li></ul></li></ul></div>

In [1]:
import pandas as pd

In [2]:
main_dir = "/u/kaimibk/Documents/Research/NAC/"
data_dir = main_dir+"data/"
out_dir = main_dir+"output/"

## Merging Data from Jackson et al (2018)

### Parsing Table 2

In [3]:
df_j1 = pd.read_csv(data_dir+"MN_17_3616_MJ_Table2.csv")
# save units row
j1_units = df_j1.loc[0]
# drop units row
df_j1 = df_j1.drop(0)
# Set index to "Name" column
df_j1 = df_j1.set_index("Name")

### Parsing Table 4

In [4]:
df_j2 = pd.read_csv(data_dir+"MN_17_3616_MJ_Table4.csv")
# save units row
j2_units = df_j2.loc[0]
# drop units row
df_j2 = df_j2.drop(0)
# drop repeat columns
df_j2 = df_j2.drop(columns=["PERIOD", "logL/Lo"])
# Set index to "Name" column
df_j2 = df_j2.set_index("Name")

### Merging and Unit Correction

In [5]:
from astropy.coordinates import SkyCoord
import astropy.units as u

In [6]:
df = pd.concat([df_j1, df_j2], axis=1)

In [7]:
temp = SkyCoord(ra=df.RA, dec=df.Dec, unit=(u.hourangle, u.deg))
df.RA, df.Dec = temp.ra.degree, temp.dec.degree
del temp

In [8]:
'''
If the VSINI is an upper limit (containing "<"), we shall use a flat distribution,
other wise we assume a uniform distribution centered at VSINI with std_dev = EVSINI.
So I am adding a column to reflect that for future reference.
'''
df["distribution"] = df.VSINI.apply(lambda x: "flat" if "<" in x else "uniform")

In [9]:
df.to_csv(data_dir+"Pleiades_master.csv")

## Query GAIA DR2 and 2MASS

In [4]:
df = pd.read_csv(data_dir+"Pleiades_master.csv")

In [11]:
from Queries import query_survey

Created TAP+ (v1.0.1) - Connection:
	Host: gea.esac.esa.int
	Use HTTPS: False
	Port: 80
	SSL Port: 443


In [12]:
gaia_features = [
        "parallax", "parallax_error",
        "radius_val", "radius_percentile_lower", "radius_percentile_upper",
        "lum_val", "lum_percentile_lower", "lum_percentile_upper",
        "teff_val", "teff_percentile_lower", "teff_percentile_upper",
        "bp_rp", "phot_g_mean_mag"
    ]
TMASS_features = [
        "j_m", "j_msigcom", 
        "h_m", "h_msigcom",
        "k_m", "k_msigcom"
   ]

In [13]:
def query(row):
    if row % 10 == 0:
        print("Inspecting row {}".format(row))
    global gaia_features
    global TMASS_features
    
    query_survey(df, row, features=gaia_features, survey='GAIA')
    query_survey(df, row, features=TMASS_features, survey='2MASS')

In [16]:
N = len(df)
for i in range(N):
    query(i)

Inspecting row 0


Inspecting row 10
Inspecting row 20
Inspecting row 30
Inspecting row 40
Inspecting row 50
Inspecting row 60
Inspecting row 70
Inspecting row 80
Inspecting row 90
Inspecting row 100
Inspecting row 110
Inspecting row 120
Inspecting row 130
Inspecting row 140
Inspecting row 150
Inspecting row 160
Inspecting row 170
Inspecting row 180
Inspecting row 190
Inspecting row 200
Inspecting row 210
Inspecting row 220
Inspecting row 230
Inspecting row 240
Inspecting row 250
Inspecting row 260
Inspecting row 270
Inspecting row 280
Inspecting row 290
Inspecting row 300
Inspecting row 310


In [18]:
df.tail(5)

,Name,RA,Dec,K2MASS,(V-K)o,Period,Ref,BCK,logL/Lo,M/Mo,...,teff_percentile_lower,teff_percentile_upper,bp_rp,phot_g_mean_mag,j_m,j_msigcom,h_m,h_msigcom,k_m,k_msigcom
314,J03511685+2349357,57.820233,23.826606,9.96,2.33,7.947,1,2.00,-0.62,0.80,...,4892.000000,5084.000000,1.245935,12.095963,10.616,0.021,10.077,0.029,9.957,0.021
315,J03512213+2509580,57.842225,25.166136,10.66,3.51,7.569,1,2.49,-1.10,0.62,...,3879.803467,4446.470215,1.820186,13.620668,11.493,0.021,10.828,0.015,10.655,0.021
316,J03512440+2405147,58.348742,24.065061,10.27,3.14,5.469,3,2.37,-0.89,0.69,...,5240.000000,5772.000000,1.009373,11.162713,9.949,0.022,9.564,0.027,9.439,0.021
317,J03512993+2353572,58.348742,24.065061,10.30,3.06,0.884,3,2.34,-0.89,0.70,...,5240.000000,5772.000000,1.009373,11.162713,9.949,0.022,9.564,0.027,9.439,0.021
318,J03514044+2458594,57.918500,24.983186,9.14,1.67,3.370,1,1.59,-0.13,1.01,...,5298.000000,5625.000000,0.916104,10.697075,9.535,0.021,9.219,0.015,9.137,0.022


In [5]:
df["radius_sigma"] = pd.DataFrame([df["radius_percentile_upper"] - df["radius_val"],
                                 df["radius_val"] - df["radius_percentile_lower"]]).mean(axis=0)

df["lum_sigma"] = pd.DataFrame([df["lum_percentile_upper"] - df["lum_val"],
                                 df["lum_val"] - df["lum_percentile_lower"]]).mean(axis=0)

df["teff_sigma"] = pd.DataFrame([df["teff_percentile_upper"] - df["teff_val"],
                                 df["teff_val"] - df["teff_percentile_lower"]]).mean(axis=0)

In [6]:
df.to_csv(data_dir+"Pleiades_master.csv")

## Isoclassify $R_*$ Calculations

### Convert Data to Isoclassify readable format

In [30]:
df = pd.read_csv(data_dir+"Pleiades_master.csv")
df["band"] = "kmag"
df["parallax"] = df["parallax"] * 10 ** (-3)
df["parallax_error"] = df["parallax_error"] * 10 ** (-3)
df["Index"] = df.index
df_iso = df[["Index", "RA", "Dec", 
              "parallax", 
              "parallax_error",
              "j_m", "j_msigcom",
              "h_m", "h_msigcom",
              "k_m", "k_msigcom", "band", "Name"
             ]]
df_iso.columns = ["id_starname", "ra","dec",
                  "parallax","parallax_err",
                  "jmag","jmag_err","hmag",
                  "hmag_err","kmag","kmag_err",
                  "band","comments"]
df = df.drop(columns="Index")
df_iso.to_csv(main_dir+"Pleiades_iso.csv")
df.index = df.Name

### Executing Isoclassify

In [31]:
import sys
from os.path import isdir, isfile
sys.path.append("/u/kaimibk/Documents/scripts/isoclassify/")
from isoclassify import *

In [32]:
k = 0
for idx, row in df.iterrows():
    out_dir = main_dir+"output/{}/".format(idx)
    
    if isdir(out_dir) == False:
        os.makedirs(out_dir)
    if isfile(out_dir+"output.csv"):
        df2 = pd.read_csv(out_dir+"output.csv")
        ## retrieve radius
        df.at[idx, "iso_rad"] = float(df2.loc[df2['id_starname'] == 'dir_rad'][str(k)].values)
        df.at[idx, "iso_rad_Err"] = float(df2.loc[df2['id_starname'] == 'dir_rad_err1'][str(k)].values)
        ## retrieve teff
        df.at[idx, "iso_teff"] = float(df2.loc[df2['id_starname'] == 'dir_teff'][str(k)].values)
        df.at[idx, "iso_teff_Err"] = float(df2.loc[df2['id_starname'] == 'dir_teff_err1'][str(k)].values)
    
    else:
        kwargs={'method': 'direct', 
                'id_starname': i, 
                'csv': main_dir+"Pleiades_iso.csv", 
                'outdir': out_dir, 'dust': 'allsky'}
        run(**kwargs);
    
        df2 = pd.read_csv(out_dir+"output.csv")
        df2.columns = range(len(columns))
        
        row["iso_rad"] = float(df2.loc[df2['id_starname'] == 'dir_rad'][str(k)].values)
        row["iso_rad_Err"] = float(df2.loc[df2['id_starname'] == 'dir_rad_err1'][str(k)].values)
        
    k += 1

In [33]:
print("The following radii could not be estimated...")
print("\t {}".format(df.index[df.iso_rad == float(0)].values))

The following radii could not be estimated...
	 ['J03424239+2320218' 'J03440167+2351462' 'J03442065+2415109'
 'J03461984+2459015' 'J03471528+2506554' 'J03473345+2341330'
 'J03474785+2513345' 'J03481608+2335155' 'J03491640+2403491'
 'J03501225+2359453']


In [34]:
df.head()

,Name,RA,Dec,K2MASS,(V-K)o,Period,Ref,BCK,logL/Lo,M/Mo,...,k_m,k_msigcom,iso_rad,iso_rad_Err,radius_sigma,lum_sigma,teff_sigma,band,iso_teff,iso_teff_Err
Name,,,,,,,,,,,,,,,,,,,,,
J03394633+2358531,J03394633+2358531,54.943067,23.981442,11.83,4.89,1.138,1,2.73,-1.66,0.41,...,11.830,0.022,0.328400,0.016717,NaN,NaN,358.076660,kmag,4222.314638,100.0
J03414664+2301198,J03414664+2301198,55.444367,23.022186,13.16,5.63,0.294,1,2.81,-2.23,0.19,...,13.162,0.028,0.268599,0.009612,NaN,NaN,NaN,kmag,3951.132399,100.0
J03414895+2303235,J03414895+2303235,55.453963,23.056539,13.19,6.09,0.239,1,2.86,-2.26,0.19,...,13.194,0.033,0.269544,0.014740,NaN,NaN,NaN,kmag,3875.296192,100.0
J03415671+2358434,J03415671+2358434,55.486317,23.978728,13.25,5.76,0.401,1,2.82,-2.27,0.18,...,13.250,0.029,0.303899,0.012164,NaN,NaN,NaN,kmag,3788.293001,100.0
J03415864+2257020,J03415864+2257020,55.494367,22.950556,11.90,4.78,6.842,1,2.72,-1.68,0.40,...,11.902,0.020,0.325585,0.016751,NaN,NaN,249.679932,kmag,4135.851442,100.0


In [35]:
df = df.drop(columns=["band", "Name"])
df.to_csv(data_dir+"Pleiades_master.csv")